# Data Challenge 13 — Interpreting Logistic Regression 

**Purpose**  
Apply what you learned about logistic regression interpretation by analyzing NYC Restaurant Inspection data. 
 
You’ll practice interpreting **continuous**, **binary**, and **categorical** predictors, compute **odds ratios**, and assess model accuracy. 

**Learning Goals**
- Convert coefficients to odds ratios using `np.exp()`.  
- Interpret ORs for continuous, binary, and categorical predictors.  
- Use accuracy to assess logistic regression performance.  
- Communicate results clearly and responsibly.  

**Data:** June 1, 2025 - Nov 4, 2025 Restaurant Health Inspection

[Restaurant Health Inspection](https://data.cityofnewyork.us/Health/DOHMH-New-York-City-Restaurant-Inspection-Results/43nn-pn8j/about_data)


## Instructor Guidance

**Hint: Use the Lecture Deck, Canvas Reading, and Docs to help you with the code**

Use this guide live; students implement below.

**Docs (Quick Links)**
- LogisticRegression — https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html  
- accuracy_score — https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html  
- OneHotEncoder — https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html  
- StandardScaler — https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html  
- np.exp — https://numpy.org/doc/stable/reference/generated/numpy.exp.html  

**Pseudocode Plan**

1️⃣ Load cleaned restaurant inspection data from the previous challenge.  
2️⃣ Define target = `IS_A` (1 = Grade A, 0 = otherwise).  
3️⃣ Predictors →  
    • Continuous = `SCORE`  
    • Binary = `CRITICAL_NUM`  
    • Categorical = `BORO`  
4️⃣ Scale continuous variables; encode categorical ones.  
5️⃣ Fit `LogisticRegression`.  
6️⃣ Exponentiate coefficients (np.exp()) → odds ratios.  
7️⃣ Interpret one continuous, one binary, and one categorical coefficient.  
8️⃣ Evaluate accuracy.  
9️⃣ Reflect on scaling choices and communication of odds.  


## You Do — Student Section
Work in pairs. Comment your choices briefly. Keep code simple—only coerce the columns you use.

## Step 1 — Imports and Plot Defaults

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


### Step 2 — Load CSV, Create Columns, Preview

- Point to your New York City Restaurant Inspection Data 
- Create the `is_A` and `critical_num` columns like you did in L11 notebook

In [2]:
df = pd.read_csv('/Users/Marcy_Student/Desktop/Marcy_Lab/DA2025_Lectures/Mod6/data/restaurant_inspection_cleaned.csv')
display(df.head())
display(df.info())
display(df.describe())

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE_DESCRIPTION,INSPECTION_DATE,ACTION,...,INSPECTION_TYPE,Latitude,Longitude,Community_Board,Council_District,Census_Tract,BIN,BBL,NTA,Location
0,50141498,DMM BAKERY,Brooklyn,6802,BAY PARKWAY,11204.0,7183314372,Chinese,2025-06-16,No violations were recorded at the time of thi...,...,Cycle Inspection / Initial Inspection,40.612100,-73.983252,311.0,47.0,25800.0,3135132.0,3.055800e+09,BK28,POINT (-73.983252132334 40.612100428335)
1,50115119,JANNAT ADEN RESTAURANT,Bronx,2620,AVENUE Z,NaN,7185004894,Middle Eastern,2025-07-09,Violations were cited in the following area(s).,...,Cycle Inspection / Initial Inspection,0.000000,0.000000,NaN,NaN,NaN,NaN,2.000000e+00,NaN,NaN
2,50121689,BELLA ITALY PIZZA,Bronx,1941,SOUTHERN BOULEVARD,10460.0,7183789577,Pizza,2025-06-09,No violations were recorded at the time of thi...,...,Administrative Miscellaneous / Initial Inspection,40.842262,-73.885759,206.0,15.0,36502.0,2010160.0,2.029600e+09,BX17,POINT (-73.885758684936 40.842261957703)
3,50142981,PIZZA PLUS,Manhattan,2253,3 AVENUE,10035.0,2122892400,Pizza,2025-06-09,Violations were cited in the following area(s).,...,Cycle Inspection / Initial Inspection,40.802072,-73.936988,111.0,8.0,19400.0,1054626.0,1.017870e+09,MN34,POINT (-73.936988471232 40.802071782442)
4,50139126,WONDER,Brooklyn,310,SCHERMERHORN STREET,11217.0,9142614549,Fusion,2025-06-07,No violations were recorded at the time of thi...,...,Inter-Agency Task Force / Initial Inspection,40.687480,-73.982245,302.0,33.0,4100.0,3000556.0,3.001728e+09,BK38,POINT (-73.982245142975 40.687480172953)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41100 entries, 0 to 41099
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   CAMIS                  41100 non-null  int64  
 1   DBA                    41100 non-null  object 
 2   BORO                   41100 non-null  object 
 3   BUILDING               40940 non-null  object 
 4   STREET                 41100 non-null  object 
 5   ZIPCODE                40597 non-null  float64
 6   PHONE                  41100 non-null  object 
 7   CUISINE_DESCRIPTION    41100 non-null  object 
 8   INSPECTION_DATE        41100 non-null  object 
 9   ACTION                 41100 non-null  object 
 10  VIOLATION_CODE         40591 non-null  object 
 11  VIOLATION_DESCRIPTION  40591 non-null  object 
 12  CRITICAL_FLAG          41100 non-null  object 
 13  SCORE                  39329 non-null  float64
 14  GRADE                  22556 non-null  object 
 15  GR

None

,CAMIS,ZIPCODE,SCORE,Latitude,Longitude,Community_Board,Council_District,Census_Tract,BIN,BBL
count,4.110000e+04,40597.000000,39329.000000,40993.000000,40993.000000,40423.000000,40423.000000,40423.000000,4.021300e+04,4.088100e+04
mean,4.845420e+07,10714.489741,29.159196,40.274106,-73.112144,256.961631,20.356530,30462.868194,2.606248e+06,2.494582e+09
std,3.473063e+06,595.241082,21.543035,4.281550,7.772006,130.928956,15.484138,31846.096493,1.362256e+06,1.346751e+09
min,4.035602e+07,10001.000000,0.000000,0.000000,-74.248708,101.000000,1.000000,100.000000,1.000000e+06,1.000000e+00
25%,5.003592e+07,10023.000000,13.000000,40.688945,-73.988314,106.000000,4.000000,8100.000000,1.049916e+06,1.010710e+09
50%,5.010674e+07,11101.000000,25.000000,40.737404,-73.954362,302.000000,20.000000,17200.000000,3.028676e+06,3.008560e+09
75%,5.014162e+07,11233.000000,39.000000,40.760766,-73.889191,402.000000,34.000000,44100.000000,4.017574e+06,4.006920e+09
max,5.017819e+07,11694.000000,203.000000,40.912822,0.000000,503.000000,51.000000,161700.000000,5.799501e+06,5.270001e+09


In [4]:
df.columns = df.columns.str.strip().str.replace(' ', '_').str.replace('-', '_')
df['INSPECTION_DATE'] = pd.to_datetime(df['INSPECTION_DATE'])
df = df[(df['INSPECTION_DATE'] >= '2025-06-01') & (df['INSPECTION_DATE'] <= '2025-11-04')]

In [5]:
# Coerce fare, tip, distance to numeric safely
num_cols = ['GRADE', 'SCORE']
df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=num_cols)

In [6]:
df['SCORE'].info()

<class 'pandas.core.series.Series'>
Index: 22554 entries, 0 to 41099
Series name: SCORE
Non-Null Count  Dtype  
--------------  -----  
22554 non-null  float64
dtypes: float64(1)
memory usage: 352.4 KB


In [7]:
df['is_A'] = (df['GRADE'] == 'A').astype(int)

In [8]:
df['is_A'].value_counts()

is_A
1    11905
0    10649
Name: count, dtype: int64

In [9]:
df['CritiCal_num'] = (df['CRITICAL_FLAG'] == 'Critical').astype(int)

In [12]:
df['CritiCal_num'].value_counts()

CritiCal_num
1    11679
0    10875
Name: count, dtype: int64

## Step 3 — Define Predictors & Target

- Target is `is_A` 
- X predictors are: SCORE, CRITICAL_NUM (created in Step 2), BORO


In [ ]:
df['BORO'] 

0         Brooklyn
5         Brooklyn
8         Brooklyn
11       Manhattan
12          Queens
           ...    
41094    Manhattan
41096       Queens
41097    Manhattan
41098       Queens
41099     Brooklyn
Name: BORO, Length: 22554, dtype: object

In [14]:
from sklearn.preprocessing import OneHotEncoder
# Dummy Variables (Method: sklearn.OneHotEncoder)
# We fit the encoder on the 'BORO' column
ohe = OneHotEncoder(drop='first', sparse_output=False)
feature_array = ohe.fit_transform(df[['BORO']])
feature_labels = list(ohe.get_feature_names_out())
dummies_skl = pd.DataFrame(feature_array, columns=feature_labels)

In [15]:
dummies_skl.head()

,BORO_Brooklyn,BORO_Manhattan,BORO_Queens,BORO_Staten Island
0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0
4,0.0,0.0,1.0,0.0


In [16]:
# Concatenate back to the original dataframe
df = pd.concat([df.reset_index(drop=True), dummies_skl.reset_index(drop=True)], axis=1)
print("\n--- Data with sklearn.OneHotEncoder ---")
df.head()


--- Data with sklearn.OneHotEncoder ---


,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE_DESCRIPTION,INSPECTION_DATE,ACTION,...,BIN,BBL,NTA,Location,is_A,CritiCal_num,BORO_Brooklyn,BORO_Manhattan,BORO_Queens,BORO_Staten Island
0,50141498,DMM BAKERY,Brooklyn,6802,BAY PARKWAY,11204.0,7183314372,Chinese,2025-06-16,No violations were recorded at the time of thi...,...,3135132.0,3.055800e+09,BK28,POINT (-73.983252132334 40.612100428335),1,0,1.0,0.0,0.0,0.0
1,50098804,GYRO EXPRESS,Brooklyn,3160,CONEY ISLAND AVENUE,11235.0,7187699228,Middle Eastern,2025-10-14,Violations were cited in the following area(s).,...,3245015.0,3.086780e+09,BK19,POINT (-73.959713615942 40.578835975614),0,0,1.0,0.0,0.0,0.0
2,50154039,AMAR BARI,Brooklyn,1075,LIBERTY AVENUE,11208.0,9172507577,Bangladeshi,2025-08-27,Violations were cited in the following area(s).,...,3093593.0,3.041710e+09,BK83,POINT (-73.869546373377 40.678346807646),0,0,1.0,0.0,0.0,0.0
3,50124438,TIPSY SHANGHAI,Manhattan,594,3 AVENUE,10016.0,2124666488,Chinese,2025-06-27,Establishment re-opened by DOHMH.,...,1019147.0,1.008940e+09,MN20,POINT (-73.975981402783 40.748698335692),0,0,0.0,1.0,0.0,0.0
4,41395494,CITI FIELD SUITE KITCHEN,Queens,000,126TH ST & ROOSEVELT AVENUE,NaN,7185958100,American,2025-09-13,Violations were cited in the following area(s).,...,NaN,4.000000e+00,NaN,NaN,1,0,0.0,0.0,1.0,0.0


## Step 4 — Split Data (70/30 Stratify by Target)

In [26]:
from sklearn.preprocessing import StandardScaler
X = df[['SCORE', 'CritiCal_num', 'BORO_Manhattan', 'BORO_Queens', 'BORO_Staten Island', 'BORO_Brooklyn']]
y = df['is_A']
X = sm.add_constant(X)
scaler = StandardScaler()
scaler.fit(X)
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

## Step 5 – Preprocessing (You can chose to do this in a Pipeline)  

- Scale continuous features  
- Pass binary as is  
- One-hot encode categorical feature (`BORO`)  

### All this steps were done above in step 4 ☝︎

## Step 6 – Fit Model & Evaluate Accuracy

- Fit `is_A ~ score` using **LogisticRegression**  
- Compute predictions with `.predict()`  
- Evaluate accuracy with `accuracy_score()`

In [27]:
# Initialize and fit the model
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

# Make predictions on the test set
y_pred = log_reg.predict(X_test)
# Print a simple accuracy score (we will learn better metrics later)
print(f"Model Accuracy: {accuracy_score(y_test, (y_pred >= 0.5).astype(int)):.2f}")

Model Accuracy: 0.97


In [39]:
log_reg.coef_[0][2]

np.float64(0.4309017704947399)

## Step 7 – Extract Coefficients and Convert to Odds Ratios


In [ ]:
coef_Score = log_reg.coef_[0][0]
intercept = log_reg.intercept_[0]

print(f"Intercept (Log-Odds): {intercept:.4f}")
print(f"Coefficient (Log-Odds): {coef_Score:.4f}")
# --- Convert to Odds Ratios ---
baseline_odds = np.exp(intercept)

odds_ratio_score = np.exp(coef_Score)
odds_ratio_CritiCal = np.exp(log_reg.coef_[0][1])
odds_ratio_Boro_Manhattan = np.exp(log_reg.coef_[0][2])
odds_ratio_Boro_Queens = np.exp(log_reg.coef_[0][3])
odds_ratio_Boro_Staten_Island = np.exp(log_reg.coef_[0][4])
odds_ratio_Boro_Brooklyn = np.exp(log_reg.coef_[0][5])

print(f"\nBaseline Odds: {baseline_odds:.4f}")

print(f"Odds Ratio for 'SCORE': {odds_ratio_score:.4f}")
print(f"Odds Ratio for 'CritiCal_num': {odds_ratio_CritiCal:.4f}")
print(f"Odds Ratio for 'BORO_Manhattan': {odds_ratio_Boro_Manhattan:.4f}")
print(f"Odds Ratio for 'BORO_Queens': {odds_ratio_Boro_Queens:.4f}")
print(f"Odds Ratio for 'BORO_Staten Island': {odds_ratio_Boro_Staten_Island:.4f}")
print(f"Odds Ratio for 'BORO_Brooklyn': {odds_ratio_Boro_Brooklyn:.4f}")   

Intercept (Log-Odds): 8.0214
Coefficient (Log-Odds): -0.0102

Baseline Odds: 3045.3660
Odds Ratio for 'SCORE': 0.9899
Odds Ratio for 'CritiCal_num': 0.6042
Odds Ratio for 'BORO_Manhattan': 1.5386
Odds Ratio for 'BORO_Queens': 1.1991
Odds Ratio for 'BORO_Staten Island': 1.0772
Odds Ratio for 'BORO_Brooklyn': 1.0267


In [38]:
df['CRITICAL_FLAG'].value_counts()


CRITICAL_FLAG
Critical          11679
Not Critical      10787
Not Applicable       88
Name: count, dtype: int64

## Step 8 – Interpret Each Predictor 

**Remember**
💡 OR > 1 → increases odds of Grade A  
💡 OR < 1 → decreases odds of Grade A

**Type markdown interpreting all 3 predictors in plain english**

- For a restaurants who has no score, the odds of them having a grade A are 3045.3660. this intercept isn't very meaningful. Make no scence!
- For Each decrease in `score`, increases the restaurant's odds of having a `grade A` increase by 1.01%.
- The odds of having grade A for restaurants on critical are 0.6042 times lower than the odds for restaurants who are not on a critical flag.
- The odds of having grade A for restaurants on in Manhattan are 53.86% higher than the restaurants in other boroughs.
- The odds of having grade A for restaurants on in Queens are 19.91% higher than the restaurants in other boroughs.
- The odds of having grade A for restaurants on in Staten Island are 7.72% higher than the restaurants in other boroughs.
- The odds of having grade A for restaurants on in Brooklyn are 2.67% higher than the restaurants in other boroughs.


# We Share — Reflection & Wrap-Up

Write **one short paragraphs** (4–6 sentences). Be specific and use evidence from your notebook.

**Which predictor had the strongest relationship with getting an A grade?**  
Use the odds ratios and accuracy to support your answer.  

My logistic regression model shows that restaurants with lower inspection scores are more likely to receive an A grade, meaning that cleaner restaurants receive better grade. Restaurants with a critical flag reduce the odds of earning an A grade, those restaurants are about 40% less likely to get an A compared to others. Among boroughs, Manhattan restaurants have the highest odds of receiving an A grade, with about 54% higher odds than restaurants in other areas. Queens also shows a strong result (about 20% higher odds), followed by Staten Island (8%) and Brooklyn (3%). Overall, Manhattan location appears to be the strongest predictor of getting an A grade, while having a critical flag is the strongest negative factor.